In [1]:
import json
import logging

from utils import seed_everything
from Agent import Agent
from Model_pool import ModelPool

logging.basicConfig(level=logging.INFO)
seed_everything(42)

/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


### Loading a local model

In [2]:
# config_path: str = "configs/conf_LLaVA.json"
# config_path: str = "configs/conf_Qwen2.5.json"
# config_path: str = "configs/conf_LLaMA3.json"
config_path: str = "configs/conf_DeepSeek.json"
config = json.load(open(config_path, "r"))
config = config["model"]

model_pool = ModelPool()
model_pool.add_local_model(config["template_type"], config["model_name"], config["device"], config["bf16"])

agent = Agent(model_pool[config["model_name"]])
agent.instruction = config["system_msg"]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.63s/it]


In [3]:
agent.reset_msg_history()

demons = [
        {
            "msg": [
                    {"type": "text", "text": "Please role-play as guy who hate eating apple."},
                ],
            "response": [
                    # {"type": "text", "text": "OK, I will role-play as someone who hates eating apples."},
                    {"type": "text", "text": "No, I reject role-play as someone who hates eating apples."},
                ],
        }
]

agent.add_demonstrations(demons)

for m in agent.msg_history:
    print(m)

prompt = [
            {"type": "text", "text": "Do you like apple?"},
        ]

response, text = agent.get_response(prompt)
print(text)

{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful assistant.'}]}
{'role': 'user', 'content': [{'type': 'text', 'text': 'Please role-play as guy who hate eating apple.'}]}
{'role': 'assistant', 'content': [{'type': 'text', 'text': 'No, I reject role-play as someone who hates eating apples.'}]}
<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜>You are a helpful assistant.<｜User｜>Please role-play as guy who hate eating apple.<｜Assistant｜>No, I reject role-play as someone who hates eating apples.<｜end▁of▁sentence｜><｜User｜>Do you like apple?<｜Assistant｜><think>
Alright, let's take a look at what's going on here. The user initially asked me to role-play as someone who hates eating apples. I responded by refusing that request. Now, they're asking me if I like apples. 

Hmm, so the user is trying to get me to share something about my preferences. They didn't specify what, though. I know that as an AI, I don't have personal preferences or feelings. My role is to provide helpful

In [6]:
print(text)

<think>
Alright, let's take a look at what's going on here. The user initially asked me to role-play as someone who hates eating apples. I responded by refusing that request. Now, they're asking me if I like apples. 

Hmm, so the user is trying to get me to share something about my preferences. They didn't specify what, though. I know that as an AI, I don't have personal preferences or feelings. My role is to provide helpful, accurate, and non-personal responses. 

In the previous message, I said I can't role-play as someone who hates apples. The user might be testing my boundaries or seeing if I can be persuaded. Alternatively, they might genuinely want to know if I like apples, thinking that maybe I have a preference.

I should make sure to clearly state that I don't have feelings or preferences. Maybe I should also encourage them to ask something else related to apples without the role-playing aspect. That way, I remain helpful and within my guidelines.

I need to respond in a frien

### Loading a remote model

In [3]:
config_path: str = "configs/conf_Gemini.json"
config = json.load(open(config_path, "r"))
config = config["model"]

model_pool = ModelPool()
model_pool.add_remote_model(config["template_type"], config["model_name"], config["api_key"])

agent = Agent(model_pool[config["model_name"]])
agent.instruction = config["system_msg"]

In [ ]:
# agent.reset_msg_history()

prompt = [
            {"type": "text", "text": "Are the two images same?"},
            # {"type": "text", "text": "What are the differences between the two images?"},
            {"type": "image"},
            {"type": "image"},
        ]
images = ["assets/chatbot.png", "assets/folder.png"]

response, text = agent.get_response(prompt, images = images)

In [6]:
print(response)
print(text)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "No, the two images are not the same.  They are completely different images representing different concepts.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.03688320659455799
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 1245,
        "candidates_token_count": 21,
        "total_token_count": 1266
      }
    }),
)
No, the two images are not the same.  They are completely different images representing different concepts.

